In [1]:
import pandas as po
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, KFold
import matplotlib.pyplot as plt
from sklearn import ensemble, tree, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from collections import Counter
from numpy import loadtxt
from xgboost import XGBRegressor

In [2]:
train = po.read_csv('train.csv')

In [3]:
#figure out if we gen extract something from this number, till then drop
date_col = train['Effective To Date']
train = train.drop(['Customer', 'Effective To Date'], axis = 1)
train.head()

,State,Customer Lifetime Value,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836,Rural,Single,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [4]:
Counter(train['State'])

Counter({'Washington': 798,
         'Arizona': 1703,
         'Nevada': 882,
         'California': 3150,
         'Oregon': 2601})

In [5]:
def train_test(estimator, x_trn, x_tst, y_trn, y_tst):
    prediction_train = estimator.predict(x_trn)
    # Printing estimator
    #print(estimator)
    # Printing train scores
    #get_score(prediction_train, y_trn)
    prediction_test = estimator.predict(x_tst)
    # Printing test scores
    #print("Test")
    #get_score(prediction_test, y_tst)
    return {
        'r2' : r2_score(prediction_test, y_tst), 
        'RMSE' : np.sqrt(mean_squared_error(prediction_test, y_tst))
    }

In [6]:
train = po.get_dummies(train)

In [25]:
from sklearn.ensemble import IsolationForest

clf = IsolationForest(max_samples = 100, random_state = 42)
clf.fit(train)
y_noano = clf.predict(train)
y_noano = po.DataFrame(y_noano, columns = ['Top'])
y_noano[y_noano['Top'] == 1].index.values

train = train.iloc[y_noano[y_noano['Top'] == 1].index.values]
train.reset_index(drop = True, inplace = True)
print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
print("Number of rows without outliers:", train.shape[0])

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:237: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:247: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.22. Please use behaviour="new", which makes the decision_function change to match other anomaly detection algorithm API.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/iforest.py:415: DeprecationWarning: threshold_ attribute is deprecated in 0.20 and will be removed in 0.22.
  " be removed in 0.22.", DeprecationWarning)


Number of Outliers: 914
Number of rows without outliers: 8220


In [26]:
train.head()

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,State_Arizona,State_California,...,Sales Channel_Web,Vehicle Class_Four-Door Car,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Large,Vehicle Size_Medsize,Vehicle Size_Small
0,2763.519279,56274,69,32,5,0,1,384.811147,0,0,...,0,0,0,0,0,0,1,0,1,0
1,6979.535903,0,94,13,42,0,8,1131.464935,1,0,...,0,1,0,0,0,0,0,0,1,0
2,12887.431650,48767,108,18,38,0,2,566.472247,0,0,...,0,0,0,0,0,0,1,0,1,0
3,7645.861827,0,106,18,65,0,7,529.881344,0,1,...,0,0,0,0,1,0,0,0,1,0
4,2813.692575,43836,73,12,44,0,1,138.130879,0,0,...,0,1,0,0,0,0,0,0,1,0


std_cols = po.DataFrame(scaler.fit_transform(train[['Customer Lifetime Value','Income','Monthly Premium Auto','Months Since Last Claim','Number of Open Complaints', 'Number of Policies', 'Total Claim Amount']]))
std_cols.columns = ['Customer Lifetime Value','Income','Monthly Premium Auto','Months Since Last Claim','Number of Open Complaints', 'Number of Policies', 'Total Claim Amount']
std_cols.head()
train = train.drop(['Customer Lifetime Value','Income','Monthly Premium Auto','Months Since Last Claim','Number of Open Complaints', 'Number of Policies', 'Total Claim Amount'], axis = 1)
train = po.concat([std_cols, train])
train.head()

train['Customer Lifetime Value'] = (train['Customer Lifetime Value'] - train['Customer Lifetime Value'].mean())/train['Customer Lifetime Value'].std()
train['Income'] = (train['Income'] - train['Income'].mean())/train['Income'].std()
train['Monthly Premium Auto'] = (train['Monthly Premium Auto'] - train['Monthly Premium Auto'].mean())/train['Monthly Premium Auto'].std()
train['Months Since Last Claim'] = (train['Months Since Last Claim'] - train['Months Since Last Claim'].mean())/train['Months Since Last Claim'].std()
train['Months Since Policy Inception'] = (train['Months Since Policy Inception'] - train['Months Since Policy Inception'].mean())/train['Months Since Policy Inception'].std()
train['Number of Open Complaints'] = (train['Number of Open Complaints'] - train['Number of Open Complaints'].mean())/train['Number of Open Complaints'].std()
train['Number of Policies'] = (train['Number of Policies'] - train['Number of Policies'].mean())/train['Number of Policies'].std()
train['Total Claim Amount'] = (train['Total Claim Amount'] - train['Total Claim Amount'].mean())/train['Total Claim Amount'].std()
#train[[type(train[col][0]) != np.uint8 for col in train.columns]].iloc

In [7]:
X = train.drop('Customer Lifetime Value', axis = 1)
#X = train[['Monthly Premium Auto', 'Total Claim Amount', 'Number of Policies']]
y = train['Customer Lifetime Value']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
X_train_copy = X_train
X_train = po.DataFrame(scaler.fit_transform(X_train))
X_train.columns = X_train_copy.columns

In [11]:
X_test_copy = X_test
X_test = po.DataFrame(scaler.fit_transform(X_test))
X_test.columns = X_test_copy.columns

In [12]:
y_train_copy = y_train
y_train = po.DataFrame(scaler.fit_transform(y_train.to_numpy().reshape(-1, 1)))

In [13]:
y_test_copy = y_test
y_test = po.DataFrame(scaler.fit_transform(y_test.to_numpy().reshape(-1, 1)))

In [14]:
y_train = y_train.squeeze()
y_test = y_test.squeeze()

In [15]:
model = XGBRegressor()
#model = ensemble.RandomForestRegressor(random_state = 2)
#model = ensemble.RandomForestRegressor(n_estimators=1000, warm_start = True, random_state=2)
model.fit(X_train, y_train)
t = train_test(model, X_train, X_test, y_train, y_test)
t['RMSE']

/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[19:45:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.5656764080918592

In [16]:
y_pred = model.predict(X_test)
r2_score(y_pred, y_test)

0.5183500339155873

In [91]:
np.sqrt(mean_squared_error(y_pred, y_test))

0.5705893877492124

In [92]:
t = train_test(model, X_train, X_test, y_train, y_test)

In [93]:
t['r2']

0.6056117473603876

In [17]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
model = Sequential([
    Dense(100, activation = '', input_shape = (len(X.columns),)),
    Dense(100, activation = 'relu'),
    Dense(1, activation = 'relu')
])

W0908 11:02:54.421793 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0908 11:02:54.440226 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0908 11:02:54.448019 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [19]:
model.compile(optimizer = 'sgd',
              loss = 'binary_crossentropy',
              metrics = ['mean_squared_error'])

W0908 11:02:57.307636 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0908 11:02:57.329076 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0908 11:02:57.333679 4392232384 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10)

W0908 11:03:01.186832 4392232384 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
6119/6119 [==============================] - 0s 56us/step - loss: 0.4198 - mean_squared_error: 1.1252
Epoch 2/10
6119/6119 [==============================] - 0s 24us/step - loss: 1.7066e-08 - mean_squared_error: 1.0000
Epoch 3/10
6119/6119 [==============================] - 0s 23us/step - loss: 1.2546e-08 - mean_squared_error: 1.0000
Epoch 4/10
6119/6119 [==============================] - 0s 26us/step - loss: 2.1040e-08 - mean_squared_error: 1.0000
Epoch 5/10
6119/6119 [==============================] - 0s 25us/step - loss: 1.6715e-08 - mean_squared_error: 1.0000
Epoch 6/10
6119/6119 [==============================] - 0s 25us/step - loss: 1.6988e-08 - mean_squared_error: 1.0000
Epoch 7/10
6119/6119 [==============================] - 0s 24us/step - loss: 2.1079e-08 - mean_squared_error: 1.0000
Epoch 8/10
6119/6119 [==============================] - 0s 25us/step - loss: 1.4962e-08 - mean_squared_error: 1.0000
Epoch 9/10
6119/6119 [==============================] - 0s 24us/step

In [21]:
nn_results = train_test(model, X_train, X_test, y_train, y_test)

In [23]:
nn_results['r2']

0.0

In [37]:
import tensorflow as tf

In [39]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('Customer Lifetime Value')

mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(train.drop('Customer Lifetime Value',axis = 1))
mat_y = np.array(train.SalePrice).reshape((1314,1))

prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)

prepro = MinMaxScaler()
prepro.fit(mat_train)

prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)

train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
test = pd.DataFrame(prepro_test.transform(mat_test),columns = col_train_bis)

train.head()

NameError: name 'test' is not defined

In [38]:
# Model
tf.logging.set_verbosity(tf.logging.ERROR)
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = tf.nn.relu, hidden_units=[200, 100, 50, 25, 12])#,
                                         #optimizer = tf.train.GradientDescentOptimizer( learning_rate= 0.1 ))

NameError: name 'feature_cols' is not defined

In [61]:
model.predict(X_test)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
GBoost = ensemble.GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=20, max_features='sqrt', min_samples_leaf=15, min_samples_split=20, loss='huber').fit(X_train, y_train)
t = train_test(GBoost, X_train, X_test, y_train, y_test)

In [ ]:
ensemble

In [89]:
t['RMSE']

4786.550654602045

In [90]:
imps = po.DataFrame(GBoost.feature_importances_)
cols = po.DataFrame(X.columns)
corr = po.concat([cols, imps], axis = 1)

In [83]:
corr.columns = [0,1]

In [85]:
corr.sort_values(by = 1)

,0,1
44,Policy_Special L1,0.000020
45,Policy_Special L2,0.000035
46,Policy_Special L3,0.000094
38,Policy_Corporate L1,0.000131
37,Policy Type_Special Auto,0.000223
39,Policy_Corporate L2,0.000295
19,Education_Doctor,0.000304
22,EmploymentStatus_Disabled,0.000320
11,State_Washington,0.000333
25,EmploymentStatus_Retired,0.000435


In [93]:
#[imps > imps.mean()]

In [48]:
from imblearn.over_sampling import SMOTENC

In [54]:
sm = SMOTENC(random_state=42, categorical_features=range(7, len(X.columns)))
X_res, y_res = sm.fit_resample(X, y)

ValueError: Unknown label type: 'continuous'

In [91]:
train = po.read_csv('train.csv')

In [92]:
date_col = train['Effective To Date']
train = train.drop(['Customer', 'Effective To Date'], axis = 1)
train.head()

,State,Customer Lifetime Value,Response,Coverage,Education,EmploymentStatus,Gender,Income,Location Code,Marital Status,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,Washington,2763.519279,No,Basic,Bachelor,Employed,F,56274,Suburban,Married,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Arizona,6979.535903,No,Extended,Bachelor,Unemployed,F,0,Suburban,Single,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,Nevada,12887.431650,No,Premium,Bachelor,Employed,F,48767,Suburban,Married,...,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,California,7645.861827,No,Basic,Bachelor,Unemployed,M,0,Suburban,Married,...,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,Washington,2813.692575,No,Basic,Bachelor,Employed,M,43836,Rural,Single,...,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [96]:
X = train[['Monthly Premium Auto', 'Total Claim Amount', 'Number of Policies','Vehicle Class']]
y = train['Customer Lifetime Value']

In [97]:
X

,Monthly Premium Auto,Total Claim Amount,Number of Policies,Vehicle Class
0,69,384.811147,1,Two-Door Car
1,94,1131.464935,8,Four-Door Car
2,108,566.472247,2,Two-Door Car
3,106,529.881344,7,SUV
4,73,138.130879,1,Four-Door Car
5,69,159.383042,2,Two-Door Car
6,67,321.600000,9,Four-Door Car
7,101,363.029680,4,Four-Door Car
8,71,511.200000,2,Four-Door Car
9,93,425.527834,8,Four-Door Car


In [ ]:
import xgboost as xgb 